# Ejemplo de Bosque Aleatorio

#### Importar librerias

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### Crear e iniciar sesión

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bosale").getOrCreate()

#### Cargar los datos y convertirlos a DataFrame

In [0]:
ruta = "dbfs:/FileStore/shared_uploads/jgamarramoreno@gmail.com/sample_libsvm_data.txt"

In [0]:
datos = spark.read.format("libsvm").load(ruta)

In [0]:
datos.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [0]:
datos.head()

Out[6]: Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 34

#### Dividir en conjunto de entrenamiento y prueba

In [0]:
(datosEntrenamiento,datosPrueba)= datos.randomSplit([0.7,0.3])

In [0]:
datosEntrenamiento.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



#### Crear y entrenar el modelo de Bosque Aleatorio

In [0]:
ba = RandomForestClassifier(labelCol="label",featuresCol="features",numTrees=50)

Entrenar el modelo

In [0]:
modelo = ba.fit(datosEntrenamiento)

#### Hacer predicciones

In [0]:
predicciones = modelo.transform(datosPrueba)

In [0]:
predicciones.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [0]:
predicciones.select("prediction","label","features").show(10)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[95,96,97,12...|
|       0.0|  0.0|(692,[122,123,148...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[125,126,127...|
|       0.0|  0.0|(692,[126,127,128...|
|       0.0|  0.0|(692,[126,127,128...|
|       0.0|  0.0|(692,[126,127,128...|
|       0.0|  0.0|(692,[127,128,129...|
|       1.0|  1.0|(692,[100,101,102...|
+----------+-----+--------------------+
only showing top 10 rows



#### Evaluar el modelo

In [0]:
evaluador = MulticlassClassificationEvaluator(labelCol="label",
                                              predictionCol="prediction",
                                              metricName="accuracy")

In [0]:
exactitud = evaluador.evaluate(predicciones)

In [0]:
print("Exactitud (accuracy) = %.2f"%(exactitud))
print("Error de Prueba = %.2f"%(1.0-exactitud))

Exactitud (accuracy) = 1.00
Error de Prueba = 0.00


In [0]:
modelo.featureImportances

Out[30]: SparseVector(692, {97: 0.0012, 100: 0.0013, 120: 0.0011, 131: 0.001, 155: 0.0009, 184: 0.0013, 208: 0.0105, 209: 0.0037, 236: 0.0009, 237: 0.0011, 272: 0.0226, 286: 0.005, 289: 0.0221, 290: 0.0034, 293: 0.0011, 299: 0.0011, 301: 0.0012, 302: 0.0134, 319: 0.0011, 322: 0.0182, 323: 0.0137, 327: 0.0151, 344: 0.0011, 347: 0.001, 350: 0.0022, 351: 0.0197, 352: 0.001, 356: 0.0129, 375: 0.0017, 377: 0.0213, 378: 0.0374, 379: 0.0654, 380: 0.0024, 382: 0.0029, 384: 0.0099, 386: 0.0066, 398: 0.0078, 400: 0.0138, 401: 0.0122, 405: 0.0736, 407: 0.0219, 408: 0.0012, 409: 0.0009, 410: 0.0057, 414: 0.0021, 426: 0.0128, 429: 0.0134, 433: 0.0176, 434: 0.0835, 435: 0.0051, 437: 0.0015, 440: 0.0037, 442: 0.001, 455: 0.0288, 460: 0.001, 461: 0.0364, 462: 0.0374, 467: 0.0092, 471: 0.0022, 472: 0.001, 483: 0.014, 484: 0.0211, 489: 0.0751, 490: 0.0482, 491: 0.0011, 492: 0.0034, 496: 0.001, 497: 0.0022, 498: 0.0013, 511: 0.0171, 512: 0.0141, 517: 0.0366, 518: 0.0228, 521: 0.001, 540: 0.0023, 545: 0.0

# Árboles con Gradiente Acelerado (Gradient Boosted Trees - GBTs)

In [0]:
from pyspark.ml.classification import GBTClassifier

#### Crear el modelo GBT

In [0]:
gbt = GBTClassifier(labelCol='label',featuresCol='features',maxIter=10)

#### Entrenar el modelo

In [0]:
modelo2 = gbt.fit(datosEntrenamiento)

#### Hacer predicciones

In [0]:
predicciones2 = modelo2.transform(datosPrueba)

In [0]:
predicciones2.select('prediction','label','features').show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[95,96,97,12...|
|       0.0|  0.0|(692,[122,123,148...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[125,126,127...|
+----------+-----+--------------------+
only showing top 5 rows



In [0]:
evaluador2 = MulticlassClassificationEvaluator(labelCol="label",
                                              predictionCol="prediction",
                                              metricName="accuracy")

In [0]:
exactitud2 = evaluador2.evaluate(predicciones2)

In [0]:
print("Exactitud (accuracy) = %.2f"%(exactitud2))
print("Error de Prueba = %.2f"%(1.0-exactitud2))

Exactitud (accuracy) = 1.00
Error de Prueba = 0.00
